In [1]:
pip install ptflops

  Preparing metadata (setup.py) ... done
  Created wheel for ptflops: filename=ptflops-0.7.1.2-py3-none-any.whl size=13213 sha256=ba7c89e3fc728aa8c3f924a18aca9adc6e7daad2a033e849bb82dd1ea51c5fd9
  Stored in directory: /root/.cache/pip/wheels/9d/90/07/20e8c3221349a85d63b319593e1bcbb6e0c995d2e2bcc5d775
Successfully built ptflops


In [2]:
pip install pytorch_msssim

In [3]:
import os
from google.colab import drive
from google.colab import auth
from google.auth import default

drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/dl_model/Dehaze')


import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from ptflops import get_model_complexity_info
from pytorch_msssim import ssim
from torch.utils.data import DataLoader
from collections import OrderedDict

from utils import AverageMeter, write_img, chw_to_hwc
from datasets.loader import PairLoader
from models.MixDehazeNet import *

Mounted at /content/drive/


In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--model', default='MixDehazeNet-l', type=str, help='model name')
# parser.add_argument('--num_workers', default=16, type=int, help='number of workers')
# parser.add_argument('--data_dir', default='../../data/haze/', type=str, help='path to dataset')
# parser.add_argument('--save_dir', default='../../data/haze/RESIDE-IN/saved_models/', type=str, help='path to models saving')
# parser.add_argument('--result_dir', default='../../data/haze/results/', type=str, help='path to results saving')
# parser.add_argument('--dataset', default='RESIDE-IN/', type=str, help='dataset name')
# parser.add_argument('--exp', default='indoor', type=str, help='experiment setting')
# args = parser.parse_args()

params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/haze/',
    'save_dir': '../../data/haze/RESIDE-IN/saved_models/',
    'result_dir': '../../data/haze/results/',
    'dataset': 'RESIDE-IN/'
}


In [4]:
def single(save_dir):
	state_dict = torch.load(save_dir)['state_dict']
	new_state_dict = OrderedDict()

	for k, v in state_dict.items():
		name = k[7:]
		new_state_dict[name] = v

	return new_state_dict


def test(test_loader, network, result_dir):
	PSNR = AverageMeter()
	SSIM = AverageMeter()

	torch.cuda.empty_cache()

	network.eval()

	os.makedirs(os.path.join(result_dir, 'imgs'), exist_ok=True)
	f_result = open(os.path.join(result_dir, 'results.csv'), 'w')

	for idx, batch in enumerate(test_loader):
		input = batch['source'].cuda()
		target = batch['target'].cuda()

		filename = batch['filename'][0]

		with torch.no_grad():
			output = network(input).clamp_(-1, 1)

			# [-1, 1] to [0, 1]
			output = output * 0.5 + 0.5
			target = target * 0.5 + 0.5

			psnr_val = 10 * torch.log10(1 / F.mse_loss(output, target)).item()

			_, _, H, W = output.size()
			down_ratio = max(1, round(min(H, W) / 256))		# Zhou Wang
			ssim_val = ssim(F.adaptive_avg_pool2d(output, (int(H / down_ratio), int(W / down_ratio))),
							F.adaptive_avg_pool2d(target, (int(H / down_ratio), int(W / down_ratio))),
							data_range=1, size_average=False).item()

		PSNR.update(psnr_val)
		SSIM.update(ssim_val)

		print('Test: [{0}]\t'
			  'PSNR: {psnr.val:.02f} ({psnr.avg:.02f})\t'
			  'SSIM: {ssim.val:.03f} ({ssim.avg:.03f})'
			  .format(idx, psnr=PSNR, ssim=SSIM))

		f_result.write('%s,%.02f,%.03f\n'%(filename, psnr_val, ssim_val))

		out_img = chw_to_hwc(output.detach().cpu().squeeze(0).numpy())
		write_img(os.path.join(result_dir, 'imgs', filename), out_img)

	f_result.close()

	os.rename(os.path.join(result_dir, 'results.csv'),
			  os.path.join(result_dir, '%.02f | %.04f.csv'%(PSNR.avg, SSIM.avg)))

In [ ]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test: [0]	PSNR: 43.40 (43.40)	SSIM: 0.998 (0.998)
Test: [1]	PSNR: 40.13 (41.77)	SSIM: 0.995 (0.997)
Test: [2]	PSNR: 42.96 (42.16)	SSIM: 0.998 (0.997)
Test: [3]	PSNR: 42.76 (42.31)	SSIM: 0.998 (0.997)
Test: [4]	PSNR: 42.34 (42.32)	SSIM: 0.997 (0.997)
Test: [5]	PSNR: 42.12 (42.28)	SSIM: 0.997 (0.997)
Test: [6]	PSNR: 41.66 (42.19)	SSIM: 0.997 (0.997)
Test: [7]	PSNR: 41.23 (42.07)	SSIM: 0.997 (0.997)
Test: [8]	PSNR: 40.83 (41.94)	SSIM: 0.996 (0.997)
Test: [9]	PSNR: 40.32 (41.77)	SSIM: 0.995 (0.997)
Test: [10]	PSNR: 45.18 (42.08)	SSIM: 0.998 (0.997)
Test: [11]	PSNR: 42.48 (42.12)	SSIM: 0.996 (0.997)
Test: [12]	PSNR: 45.33 (42.36)	SSIM: 0.998 (0.997)
Test: [13]	PSNR: 44.83 (42.54)	SSIM: 0.998 (0.997)
Test: [14]	PSNR: 44.53 (42.67)	SSIM: 0.998 (0.997)
Test: [15]	PSNR: 44.59 (42.79)	SSIM: 0.997 (0.997)
Test: [16]	PSNR: 44.12 (42.87)	SSIM: 0.997 (0.997)
Test: [17]	PSNR: 43.39 (42.90)	SSIM: 0.997 (0.997)
Test: [18]	PSNR: 43.58 (42.94)	SSIM: 0.997 (0.997)
Test: [19]	PSNR: 42.58 (42.92)	SSIM: 0.99

In [ ]:
params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/haze/',
    'save_dir': '../../data/haze/RESIDE-OUT/saved_models/',
    'result_dir': '../../data/haze/results/',
    'dataset': 'RESIDE-OUT/',
    'exp': 'outdoor'
}

In [ ]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l
Test: [0]	PSNR: 33.63 (33.63)	SSIM: 0.988 (0.988)
Test: [1]	PSNR: 37.82 (35.73)	SSIM: 0.991 (0.990)
Test: [2]	PSNR: 36.33 (35.93)	SSIM: 0.976 (0.985)
Test: [3]	PSNR: 33.26 (35.26)	SSIM: 0.990 (0.986)
Test: [4]	PSNR: 37.45 (35.70)	SSIM: 0.988 (0.987)
Test: [5]	PSNR: 35.51 (35.67)	SSIM: 0.989 (0.987)
Test: [6]	PSNR: 33.94 (35.42)	SSIM: 0.980 (0.986)
Test: [7]	PSNR: 33.61 (35.19)	SSIM: 0.981 (0.985)
Test: [8]	PSNR: 38.78 (35.59)	SSIM: 0.986 (0.986)
Test: [9]	PSNR: 37.46 (35.78)	SSIM: 0.990 (0.986)
Test: [10]	PSNR: 39.01 (36.07)	SSIM: 0.993 (0.987)
Test: [11]	PSNR: 36.89 (36.14)	SSIM: 0.982 (0.986)
Test: [12]	PSNR: 37.24 (36.22)	SSIM: 0.990 (0.987)
Test: [13]	PSNR: 37.09 (36.29)	SSIM: 0.989 (0.987)
Test: [14]	PSNR: 38.37 (36.43)	SSIM: 0.991 (0.987)
Test: [15]	PSNR: 40.78 (36.70)	SSIM: 0.990 (0.987)
Test: [16]	PSNR: 43.26 (37.08)	SSIM: 0.989 (0.987)
Test: [17]	PSNR: 39.36 (37.21)	SSIM: 0.974 (0.987)
Test: [18]	PSNR: 34.61 (37.07)	SSIM: 0

In [ ]:
params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/haze/',
    'save_dir': '../../data/haze/RESIDE-IN/saved_models/',
    'result_dir': '../../data/haze/cross-results/',
    'dataset': 'RESIDE-OUT/'
}

In [ ]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l
Test: [0]	PSNR: 19.84 (19.84)	SSIM: 0.931 (0.931)
Test: [1]	PSNR: 18.02 (18.93)	SSIM: 0.780 (0.856)
Test: [2]	PSNR: 19.92 (19.26)	SSIM: 0.885 (0.865)
Test: [3]	PSNR: 17.63 (18.85)	SSIM: 0.929 (0.881)
Test: [4]	PSNR: 23.92 (19.87)	SSIM: 0.922 (0.889)
Test: [5]	PSNR: 22.04 (20.23)	SSIM: 0.925 (0.895)
Test: [6]	PSNR: 26.77 (21.16)	SSIM: 0.941 (0.902)
Test: [7]	PSNR: 24.62 (21.60)	SSIM: 0.929 (0.905)
Test: [8]	PSNR: 24.46 (21.91)	SSIM: 0.879 (0.902)
Test: [9]	PSNR: 19.18 (21.64)	SSIM: 0.865 (0.898)
Test: [10]	PSNR: 15.66 (21.10)	SSIM: 0.847 (0.894)
Test: [11]	PSNR: 20.75 (21.07)	SSIM: 0.917 (0.896)
Test: [12]	PSNR: 26.80 (21.51)	SSIM: 0.967 (0.901)
Test: [13]	PSNR: 20.75 (21.45)	SSIM: 0.898 (0.901)
Test: [14]	PSNR: 20.34 (21.38)	SSIM: 0.904 (0.901)
Test: [15]	PSNR: 17.47 (21.14)	SSIM: 0.835 (0.897)
Test: [16]	PSNR: 26.50 (21.45)	SSIM: 0.930 (0.899)
Test: [17]	PSNR: 24.45 (21.62)	SSIM: 0.899 (0.899)
Test: [18]	PSNR: 22.87 (21.68)	SSIM: 0

In [ ]:
params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/haze/',
    'save_dir': '../../data/haze/RESIDE-OUT/saved_models/',
    'result_dir': '../../data/haze/cross-results/',
    'dataset': 'RESIDE-IN/'
}

In [ ]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l
Test: [0]	PSNR: 29.82 (29.82)	SSIM: 0.957 (0.957)
Test: [1]	PSNR: 22.08 (25.95)	SSIM: 0.902 (0.930)
Test: [2]	PSNR: 28.59 (26.83)	SSIM: 0.952 (0.937)
Test: [3]	PSNR: 27.46 (26.99)	SSIM: 0.945 (0.939)
Test: [4]	PSNR: 26.50 (26.89)	SSIM: 0.940 (0.939)
Test: [5]	PSNR: 25.52 (26.66)	SSIM: 0.933 (0.938)
Test: [6]	PSNR: 24.80 (26.39)	SSIM: 0.928 (0.937)
Test: [7]	PSNR: 24.01 (26.10)	SSIM: 0.921 (0.935)
Test: [8]	PSNR: 23.27 (25.78)	SSIM: 0.915 (0.933)
Test: [9]	PSNR: 22.59 (25.46)	SSIM: 0.907 (0.930)
Test: [10]	PSNR: 27.84 (25.68)	SSIM: 0.929 (0.930)
Test: [11]	PSNR: 18.85 (25.11)	SSIM: 0.844 (0.923)
Test: [12]	PSNR: 26.02 (25.18)	SSIM: 0.915 (0.922)
Test: [13]	PSNR: 24.46 (25.13)	SSIM: 0.902 (0.921)
Test: [14]	PSNR: 23.31 (25.01)	SSIM: 0.894 (0.919)
Test: [15]	PSNR: 22.52 (24.85)	SSIM: 0.886 (0.917)
Test: [16]	PSNR: 21.40 (24.65)	SSIM: 0.873 (0.914)
Test: [17]	PSNR: 20.66 (24.43)	SSIM: 0.864 (0.912)
Test: [18]	PSNR: 19.98 (24.19)	SSIM: 0

In [5]:
params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/rain/',
    'save_dir': '../../data/haze/RESIDE-OUT/saved_models/',
    'result_dir': '../../data/rain/cross-results/',
    'dataset': 'GTtest/'
}

In [6]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test: [0]	PSNR: 25.68 (25.68)	SSIM: 0.793 (0.793)
Test: [1]	PSNR: 25.47 (25.58)	SSIM: 0.792 (0.792)
Test: [2]	PSNR: 25.61 (25.59)	SSIM: 0.798 (0.794)
Test: [3]	PSNR: 25.44 (25.55)	SSIM: 0.791 (0.793)
Test: [4]	PSNR: 25.37 (25.52)	SSIM: 0.793 (0.793)
Test: [5]	PSNR: 25.26 (25.47)	SSIM: 0.791 (0.793)
Test: [6]	PSNR: 25.26 (25.44)	SSIM: 0.791 (0.793)
Test: [7]	PSNR: 25.71 (25.48)	SSIM: 0.790 (0.792)
Test: [8]	PSNR: 25.22 (25.45)	SSIM: 0.791 (0.792)
Test: [9]	PSNR: 25.51 (25.45)	SSIM: 0.786 (0.792)
Test: [10]	PSNR: 24.91 (25.40)	SSIM: 0.785 (0.791)
Test: [11]	PSNR: 24.93 (25.36)	SSIM: 0.787 (0.791)
Test: [12]	PSNR: 24.93 (25.33)	SSIM: 0.787 (0.790)
Test: [13]	PSNR: 24.82 (25.30)	SSIM: 0.781 (0.790)
Test: [14]	PSNR: 25.28 (25.29)	SSIM: 0.782 (0.789)
Test: [15]	PSNR: 25.32 (25.30)	SSIM: 0.778 (0.788)
Test: [16]	PSNR: 25.38 (25.30)	SSIM: 0.783 (0.788)
Test: [17]	PSNR: 25.70 (25.32)	SSIM: 0.781 (0.788)
Test: [18]	PSNR: 25.70 (25.34)	SSIM: 0.781 (0.787)
Test: [19]	PSNR: 25.46 (25.35)	SSIM: 0.77

In [7]:
params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/rain/',
    'save_dir': '../../data/haze/RESIDE-IN/saved_models/',
    'result_dir': '../../data/rain/cross-results/',
    'dataset': 'GTtest/'
}

In [8]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l
Test: [0]	PSNR: 19.81 (19.81)	SSIM: 0.724 (0.724)
Test: [1]	PSNR: 19.74 (19.77)	SSIM: 0.722 (0.723)
Test: [2]	PSNR: 19.82 (19.79)	SSIM: 0.730 (0.725)
Test: [3]	PSNR: 19.76 (19.78)	SSIM: 0.723 (0.725)
Test: [4]	PSNR: 19.95 (19.81)	SSIM: 0.729 (0.726)
Test: [5]	PSNR: 20.17 (19.87)	SSIM: 0.732 (0.727)
Test: [6]	PSNR: 20.17 (19.91)	SSIM: 0.732 (0.728)
Test: [7]	PSNR: 20.14 (19.94)	SSIM: 0.731 (0.728)
Test: [8]	PSNR: 20.00 (19.95)	SSIM: 0.731 (0.728)
Test: [9]	PSNR: 20.07 (19.96)	SSIM: 0.725 (0.728)
Test: [10]	PSNR: 20.35 (20.00)	SSIM: 0.730 (0.728)
Test: [11]	PSNR: 20.16 (20.01)	SSIM: 0.726 (0.728)
Test: [12]	PSNR: 20.16 (20.02)	SSIM: 0.726 (0.728)
Test: [13]	PSNR: 20.37 (20.05)	SSIM: 0.726 (0.728)
Test: [14]	PSNR: 20.35 (20.07)	SSIM: 0.726 (0.728)
Test: [15]	PSNR: 20.41 (20.09)	SSIM: 0.722 (0.727)
Test: [16]	PSNR: 20.67 (20.12)	SSIM: 0.732 (0.728)
Test: [17]	PSNR: 20.49 (20.14)	SSIM: 0.725 (0.727)
Test: [18]	PSNR: 20.49 (20.16)	SSIM: 0

In [9]:
params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/rain/',
    'save_dir': '../../data/haze/RESIDE-OUT/saved_models/',
    'result_dir': '../../data/rain/A2I2-results/',
    'dataset': 'HAZEtest/'
}

In [10]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l
Test: [0]	PSNR: 13.44 (13.44)	SSIM: 0.402 (0.402)
Test: [1]	PSNR: 15.17 (14.31)	SSIM: 0.514 (0.458)
Test: [2]	PSNR: 16.59 (15.07)	SSIM: 0.430 (0.449)
Test: [3]	PSNR: 16.34 (15.39)	SSIM: 0.401 (0.437)
Test: [4]	PSNR: 15.85 (15.48)	SSIM: 0.385 (0.426)
Test: [5]	PSNR: 16.44 (15.64)	SSIM: 0.522 (0.442)
Test: [6]	PSNR: 17.27 (15.87)	SSIM: 0.413 (0.438)
Test: [7]	PSNR: 11.95 (15.38)	SSIM: 0.516 (0.448)
Test: [8]	PSNR: 13.46 (15.17)	SSIM: 0.565 (0.461)
Test: [9]	PSNR: 13.25 (14.98)	SSIM: 0.482 (0.463)
Test: [10]	PSNR: 12.50 (14.75)	SSIM: 0.367 (0.454)
Test: [11]	PSNR: 12.39 (14.55)	SSIM: 0.291 (0.441)
Test: [12]	PSNR: 12.92 (14.43)	SSIM: 0.414 (0.439)
Test: [13]	PSNR: 14.34 (14.42)	SSIM: 0.418 (0.437)
Test: [14]	PSNR: 13.10 (14.33)	SSIM: 0.412 (0.435)
Test: [15]	PSNR: 15.29 (14.39)	SSIM: 0.443 (0.436)
Test: [16]	PSNR: 14.96 (14.43)	SSIM: 0.445 (0.436)
Test: [17]	PSNR: 15.26 (14.47)	SSIM: 0.458 (0.438)
Test: [18]	PSNR: 15.44 (14.52)	SSIM: 0

In [11]:
params = {
    'model': 'MixDehazeNet-l',
    'num_workers': 16,
    'data_dir': '../../data/rain/',
    'save_dir': '../../data/haze/RESIDE-IN/saved_models/',
    'result_dir': '../../data/rain/A2I2-results/',
    'dataset': 'HAZEtest/'
}

In [12]:
network = eval(params['model'].replace('-', '_'))()
network.cuda()
saved_model_dir = os.path.join(params['save_dir'], params['model']+'.pth')

if os.path.exists(saved_model_dir):
	print('==> Start testing, current model name: ' + params['model'])
	network.load_state_dict(single(saved_model_dir))
else:
	print('==> No existing trained model!')
	exit(0)

dataset_dir = os.path.join(params['data_dir'], params['dataset'])
test_dataset = PairLoader(dataset_dir, 'test', 'test')
test_loader = DataLoader(test_dataset,
							batch_size=1,
							num_workers=params['num_workers'],
							pin_memory=True)

result_dir = os.path.join(params['result_dir'], params['dataset'], params['model'])
test(test_loader, network, result_dir)

==> Start testing, current model name: MixDehazeNet-l
Test: [0]	PSNR: 9.25 (9.25)	SSIM: 0.295 (0.295)
Test: [1]	PSNR: 5.70 (7.48)	SSIM: 0.116 (0.206)
Test: [2]	PSNR: 7.35 (7.43)	SSIM: 0.215 (0.209)
Test: [3]	PSNR: 10.58 (8.22)	SSIM: 0.324 (0.238)
Test: [4]	PSNR: 9.27 (8.43)	SSIM: 0.191 (0.228)
Test: [5]	PSNR: 5.91 (8.01)	SSIM: 0.062 (0.201)
Test: [6]	PSNR: 9.86 (8.27)	SSIM: 0.240 (0.206)
Test: [7]	PSNR: 7.21 (8.14)	SSIM: 0.223 (0.208)
Test: [8]	PSNR: 6.38 (7.95)	SSIM: 0.140 (0.201)
Test: [9]	PSNR: 8.77 (8.03)	SSIM: 0.283 (0.209)
Test: [10]	PSNR: 6.89 (7.92)	SSIM: 0.140 (0.203)
Test: [11]	PSNR: 8.09 (7.94)	SSIM: 0.143 (0.198)
Test: [12]	PSNR: 8.47 (7.98)	SSIM: 0.200 (0.198)
Test: [13]	PSNR: 9.17 (8.06)	SSIM: 0.213 (0.199)
Test: [14]	PSNR: 8.06 (8.06)	SSIM: 0.238 (0.202)
Test: [15]	PSNR: 9.36 (8.15)	SSIM: 0.278 (0.206)
Test: [16]	PSNR: 8.88 (8.19)	SSIM: 0.233 (0.208)
Test: [17]	PSNR: 9.23 (8.25)	SSIM: 0.233 (0.209)
Test: [18]	PSNR: 9.30 (8.30)	SSIM: 0.234 (0.211)
Test: [19]	PSNR: 8.64 (8